In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
from pandas.tseries.offsets import MonthEnd

In [2]:
types = {'id': 'int32',
         'item_nbr': 'int32',
         'store_nbr': 'int8',
         'unit_sales': 'float32',
         'onpromotion': bool}

In [3]:
train = pd.read_csv('train.csv',usecols=['date','item_nbr','store_nbr','unit_sales','onpromotion'],\
                    parse_dates=['date'],dtype=types, infer_datetime_format=True)

/home/will/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
train = train.fillna(2,axis=1)
train.onpromotion = train.onpromotion.astype(np.int8)

In [5]:
train.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion
0,2013-01-01,25,103665,7.0,2
1,2013-01-01,25,105574,1.0,2
2,2013-01-01,25,105575,2.0,2
3,2013-01-01,25,108079,1.0,2
4,2013-01-01,25,108701,1.0,2


In [6]:
val = train[train.date > '2017-07-15']

In [7]:
train = train[train.date <= '2017-07-15']

In [8]:
def isConsecutive(seq):
    # if non-consecutive, need to re-map to consecutive number starting from 1
    uniq = pd.unique(seq)
    return len(uniq) == (uniq.max()-uniq.min() + 1)

In [9]:
def isUniqBigger(seq1,seq2):
    return set(seq1) >= set(seq2)

In [10]:
def reMapDF(df,cols):
    # remap cols in dataframe to consecutive integers starting from one
    for col in cols:
        uniq = pd.unique(df[col])
        dict_ = {item:i+1 for i,item in enumerate(uniq)}
        df = df.replace({col:dict_})
    return df

In [11]:
def dimentionDF(df,cols):
    return {col:len(set(df[col])) for col in cols}

** Data Processing **

In [12]:
holidays_events = pd.read_csv('holidays_events.csv',parse_dates=['date'],infer_datetime_format=True)

In [13]:
holidays_events2 = reMapDF(holidays_events.drop('description',1),['type', 'locale', 'locale_name'])

In [14]:
holidays_events2[['type', 'locale', 'locale_name','transferred']] = \
        holidays_events2[['type', 'locale', 'locale_name','transferred']].astype('int8')

In [15]:
items = pd.read_csv('items.csv')

In [16]:
items2 = reMapDF(items,['family','class'])

In [17]:
items2[['family','class','perishable']] = \
        items2[['family','class','perishable']].astype('int8')

In [18]:
stores = pd.read_csv('stores.csv')

In [19]:
stores2 = reMapDF(stores,['city', 'state', 'type'])

In [20]:
stores2 = stores2.astype('int8')

In [21]:
dateVar = pd.DataFrame(pd.date_range('2013-01-01', '2017-08-31'),columns=['date'])

In [22]:
dateVar['dayOfWeek'] = dateVar.date.dt.dayofweek

In [23]:
dateVar['payDay'] = ((dateVar.date.dt.day == dateVar.date.dt.days_in_month) | \
                     (dateVar.date.dt.day == 15)) * 1

In [24]:
dateVar['month'] = dateVar.date.dt.month

In [25]:
dateVar['monthSinceT0'] = (dateVar.date - pd.datetime(2013,1,1))/np.timedelta64(1, 'M')

In [26]:
dateVar['earthquake'] = (dateVar.date > '2016-04-16') & (dateVar.date <= '2016-04-24')

In [27]:
dateVar.columns.values

array(['date', 'dayOfWeek', 'payDay', 'month', 'monthSinceT0', 'earthquake'], dtype=object)

In [28]:
dateVar[['dayOfWeek', 'payDay', 'month', 'monthSinceT0','earthquake']] = \
    dateVar[['dayOfWeek', 'payDay', 'month', 'monthSinceT0','earthquake']].astype('int8')

In [29]:
oil = pd.read_csv('oil.csv',parse_dates=['date'],infer_datetime_format=True)

In [30]:
dateVar = pd.merge(dateVar,oil,'left').fillna(method='bfill')

** mergeFillCastsss is the main function the join all the tables defined above to train/test/val. After this step, we need to re-map item_nbr. **

In [31]:
def mergeFillCast(df1,df2,key):
    cols = df2.columns.values
    types = df2.dtypes.values
    dict_ = {col:type_ for col,type_ in zip(cols,types)}
    dfOut = pd.merge(df1, df2, how='left', on=key, 
             suffixes=('', '_y'), copy=True, indicator=False).fillna(0)
    dfOut[cols] = \
        dfOut[cols].astype(dict_)
    return dfOut

In [32]:
def mergeFillCastsss(df0,dfs,keys):
    for df,key in zip(dfs,keys):
        df0 = mergeFillCast(df0,df,key)
    return df0

In [33]:
dfs = [dateVar,holidays_events2,items2,stores2]
keys = ['date','date','item_nbr','store_nbr']

In [34]:
val_merge = mergeFillCastsss(val,dfs,keys)

In [35]:
pd.options.display.max_columns = 999
val_merge.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion,dayOfWeek,payDay,month,monthSinceT0,earthquake,dcoilwtico,type,locale,locale_name,transferred,family,class,perishable,city,state,type_y,cluster
0,2017-07-16,1,105574,2.0,0,6,0,7,54,0,46.02,0,0,0,0,1,6,0,1,1,1,13
1,2017-07-16,1,105575,1.0,0,6,0,7,54,0,46.02,0,0,0,0,1,6,0,1,1,1,13
2,2017-07-16,1,105857,1.0,0,6,0,7,54,0,46.02,0,0,0,0,1,9,0,1,1,1,13
3,2017-07-16,1,106716,1.0,0,6,0,7,54,0,46.02,0,0,0,0,1,10,0,1,1,1,13
4,2017-07-16,1,108698,1.0,0,6,0,7,54,0,46.02,0,0,0,0,4,14,1,1,1,1,13


In [37]:
train_merge = mergeFillCastsss(train,dfs,keys)

In [38]:
train_merge.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion,dayOfWeek,payDay,month,monthSinceT0,earthquake,dcoilwtico,type,locale,locale_name,transferred,family,class,perishable,city,state,type_y,cluster
0,2013-01-01,25,103665,7.0,2,1,0,1,0,0,93.14,1,3,15,0,3,5,1,11,10,1,1
1,2013-01-01,25,105574,1.0,2,1,0,1,0,0,93.14,1,3,15,0,1,6,0,11,10,1,1
2,2013-01-01,25,105575,2.0,2,1,0,1,0,0,93.14,1,3,15,0,1,6,0,11,10,1,1
3,2013-01-01,25,108079,1.0,2,1,0,1,0,0,93.14,1,3,15,0,1,11,0,11,10,1,1
4,2013-01-01,25,108701,1.0,2,1,0,1,0,0,93.14,1,3,15,0,4,14,1,11,10,1,1


In [39]:
item_uniq = pd.unique(train_merge.item_nbr)

In [40]:
item_dict = {item:i+1 for i,item in enumerate(item_uniq)}

In [41]:
iter_mapping = lambda x: item_dict[x] if x in item_dict else 0

In [42]:
val_merge['item_nbr'] = val_merge.item_nbr.map(iter_mapping)

In [60]:
train_merge['item_nbr'] = train_merge.item_nbr.map(iter_mapping)

In [65]:
discrete = ['store_nbr', 'item_nbr', 'onpromotion',
       'dayOfWeek', 'payDay', 'month', 'monthSinceT0', 'earthquake',
       'type', 'locale', 'locale_name', 'transferred',
       'family', 'class', 'perishable', 'city', 'state', 'type_y',
       'cluster']

In [78]:
def export2csv(dfIn,outPath,IsTrain=True):
    dfIn.loc[:,discrete].to_csv(outPath+'_discrete.csv',header=False,index=False)
    dfIn.loc[:,'dcoilwtico'].to_csv(outPath+'_continuous.csv',header=False,index=False)
    if IsTrain:
        dfIn.loc[:,'unit_sales'].to_csv(outPath+'_y.csv',header=False,index=False)
    else:
        dfIn.loc[:,'id'].to_csv(outPath+'_id.csv',header=False,index=False)

In [68]:
export2csv(val_merge,'val')

In [69]:
export2csv(train_merge,'train')

In [70]:
del train_merge, val_merge

In [71]:
test = pd.read_csv('test.csv',parse_dates=['date'],dtype=types, infer_datetime_format=True)

In [72]:
test = test.fillna(2,axis=1)
test.onpromotion = test.onpromotion.astype(np.int8)

In [73]:
test_merge = mergeFillCastsss(test,dfs,keys)

In [74]:
test_merge = test_merge.drop(['date'],1)

In [75]:
test_merge['item_nbr'] = test_merge.item_nbr.map(iter_mapping)

In [80]:
export2csv(test_merge,'test',False)

In [86]:
test_merge.head()

,id,store_nbr,item_nbr,onpromotion,dayOfWeek,payDay,month,monthSinceT0,earthquake,dcoilwtico,type,locale,locale_name,transferred,family,class,perishable,city,state,type_y,cluster
0,125497040,1,1601,0,2,0,8,55,0,46.8,0,0,0,0,1,1,0,1,1,1,13
1,125497041,1,2743,0,2,0,8,55,0,46.8,0,0,0,0,1,2,0,1,1,1,13
2,125497042,1,1386,0,2,0,8,55,0,46.8,0,0,0,0,2,3,0,1,1,1,13
3,125497043,1,1370,0,2,0,8,55,0,46.8,0,0,0,0,1,4,0,1,1,1,13
4,125497044,1,1,0,2,0,8,55,0,46.8,0,0,0,0,3,5,1,1,1,1,13


<br>

<br>

** Data exploration **

In [7]:
iterTrain = set(train.item_nbr)

In [8]:
iterTest = set(test.item_nbr)

There are 60 items in test but not in Train

In [11]:
iterTrain >= iterTest

False

In [14]:
len(iterTest - iterTrain)

60

In [15]:
storeTrain = set(train.store_nbr)

In [16]:
storeTest = set(test.store_nbr)

In [17]:
storeTrain >= storeTest

True

In [18]:
print train.date.min(),train.date.max()

2013-01-01 00:00:00 2017-08-15 00:00:00


In [19]:
print test.date.min(),test.date.max()

2017-08-16 00:00:00 2017-08-31 00:00:00


In [7]:
isConsecutive(train.store_nbr)

True

In [8]:
isConsecutive(train.item_nbr)

False

In [9]:
isUniqBigger(train.item_nbr,val.item_nbr)

False

In [13]:
holidays_events.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [14]:
dimentionDF(holidays_events,['type', 'locale', 'locale_name', 'description',
       'transferred'])

{'description': 103,
 'locale': 3,
 'locale_name': 24,
 'transferred': 2,
 'type': 6}

In [17]:
holidays_events2.dtypes

date           datetime64[ns]
type                     int8
locale                   int8
locale_name              int8
transferred              int8
dtype: object

In [18]:
holidays_events2.head()

,date,type,locale,locale_name,transferred
0,2012-03-02,1,1,1,0
1,2012-04-01,1,2,2,0
2,2012-04-12,1,1,3,0
3,2012-04-14,1,1,4,0
4,2012-04-21,1,1,5,0


In [25]:
items.head()

,item_nbr,family,class,perishable
0,96995,GROCERY I,1093,0
1,99197,GROCERY I,1067,0
2,103501,CLEANING,3008,0
3,103520,GROCERY I,1028,0
4,103665,BREAD/BAKERY,2712,1


In [28]:
items2.head()

,item_nbr,family,class,perishable
0,96995,1,1,0
1,99197,1,2,0
2,103501,2,3,0
3,103520,1,4,0
4,103665,3,5,1


In [29]:
items2.dtypes

item_nbr      int64
family         int8
class          int8
perishable     int8
dtype: object

In [24]:
dimentionDF(items,['family','class'])

{'class': 337, 'family': 33}

In [31]:
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [33]:
stores2.head()

,store_nbr,city,state,type,cluster
0,1,1,1,1,13
1,2,1,1,1,13
2,3,1,1,1,8
3,4,1,1,1,9
4,5,2,2,1,4


In [34]:
dimentionDF(stores,['city', 'state', 'type','cluster'])

{'city': 22, 'cluster': 17, 'state': 16, 'type': 5}